# Risk Mediation for Heart Failure for BMI to FPG and SBP

This notebook is designed to calculate the mediation factors for heart failure for FPG and SBP. LDL-C does not directly affect HF in our model, and so does not have a mediation factor here. We also test if we need to include the risk mediation of BMI on FPG based on the results of a trial and the observed effect on FPG.

## First, we will load in the relative risk values from GBD

In [1]:
from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws
import numpy as np
import pandas as pd

In [2]:
ihd_bmi = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=370, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_bmi = ihd_bmi.loc[ihd_bmi.cause_id==493].reset_index()
ihd_bmi = ihd_bmi[['age_group_id','sex_id','draw_0']]
ihd_bmi['age_start'] = np.where(ihd_bmi['age_group_id']==10,25,np.where(ihd_bmi['age_group_id']==11,30,np.where(ihd_bmi['age_group_id']==12,35,np.where(ihd_bmi['age_group_id']==13,40,np.where(ihd_bmi['age_group_id']==14,45,np.where(ihd_bmi['age_group_id']==15,50,np.where(ihd_bmi['age_group_id']==16,55,np.where(ihd_bmi['age_group_id']==17,60,np.where(ihd_bmi['age_group_id']==18,65,np.where(ihd_bmi['age_group_id']==19,70,np.where(ihd_bmi['age_group_id']==20,75,np.where(ihd_bmi['age_group_id']==30,80,np.where(ihd_bmi['age_group_id']==31,85,np.where(ihd_bmi['age_group_id']==32,90,0))))))))))))))
ihd_bmi['sex'] = np.where(ihd_bmi['sex_id']==1,'Male','Female')
ihd_bmi = ihd_bmi.rename(columns={"draw_0": "ihd_bmi_rr"}).reset_index()
ihd_bmi = ihd_bmi.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_bmi.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,ihd_bmi_rr,age_start,sex
0,0,0,19,1,1.296208,70,Male
1,1,1,20,1,1.291149,75,Male
2,2,2,18,1,1.395200,65,Male
3,3,3,16,1,1.425974,55,Male
4,4,4,11,1,2.094858,30,Male


In [3]:
ihd_fpg = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=105, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_fpg = ihd_fpg.loc[ihd_fpg.cause_id==493].reset_index()
ihd_fpg = ihd_fpg[['age_group_id','sex_id','draw_0']]
ihd_fpg['age_start'] = np.where(ihd_fpg['age_group_id']==10,25,np.where(ihd_fpg['age_group_id']==11,30,np.where(ihd_fpg['age_group_id']==12,35,np.where(ihd_fpg['age_group_id']==13,40,np.where(ihd_fpg['age_group_id']==14,45,np.where(ihd_fpg['age_group_id']==15,50,np.where(ihd_fpg['age_group_id']==16,55,np.where(ihd_fpg['age_group_id']==17,60,np.where(ihd_fpg['age_group_id']==18,65,np.where(ihd_fpg['age_group_id']==19,70,np.where(ihd_fpg['age_group_id']==20,75,np.where(ihd_fpg['age_group_id']==30,80,np.where(ihd_fpg['age_group_id']==31,85,np.where(ihd_fpg['age_group_id']==32,90,0))))))))))))))
ihd_fpg['sex'] = np.where(ihd_fpg['sex_id']==1,'Male','Female')
ihd_fpg = ihd_fpg.rename(columns={"draw_0": "ihd_fpg_rr"}).reset_index()
ihd_fpg = ihd_fpg.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_fpg.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,ihd_fpg_rr,age_start,sex
0,0,0,10,2,1.9325,25,Female
1,1,1,11,2,1.6915,30,Female
2,2,2,12,2,1.4505,35,Female
3,3,3,13,2,1.3135,40,Female
4,4,4,14,2,1.2805,45,Female


In [17]:
ihd_sbp = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=107, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_sbp = ihd_sbp.loc[ihd_sbp.cause_id==493].reset_index()
ihd_sbp = ihd_sbp[['age_group_id','sex_id','draw_0']]
ihd_sbp['age_start'] = np.where(ihd_sbp['age_group_id']==10,25,np.where(ihd_sbp['age_group_id']==11,30,np.where(ihd_sbp['age_group_id']==12,35,np.where(ihd_sbp['age_group_id']==13,40,np.where(ihd_sbp['age_group_id']==14,45,np.where(ihd_sbp['age_group_id']==15,50,np.where(ihd_sbp['age_group_id']==16,55,np.where(ihd_sbp['age_group_id']==17,60,np.where(ihd_sbp['age_group_id']==18,65,np.where(ihd_sbp['age_group_id']==19,70,np.where(ihd_sbp['age_group_id']==20,75,np.where(ihd_sbp['age_group_id']==30,80,np.where(ihd_sbp['age_group_id']==31,85,np.where(ihd_sbp['age_group_id']==32,90,0))))))))))))))
ihd_sbp['sex'] = np.where(ihd_sbp['sex_id']==1,'Male','Female')
ihd_sbp = ihd_sbp.rename(columns={"draw_0": "ihd_sbp_rr"}).reset_index()
ihd_sbp = ihd_sbp.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_sbp.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,ihd_sbp_rr,age_start,sex
0,0,0,10,1,2.0317,25,Male
1,1,1,11,1,1.8558,30,Male
2,2,2,12,1,1.6799,35,Male
3,3,3,13,1,1.5729,40,Male
4,4,4,14,1,1.5348,45,Male


## Now we can calculate the deltas

In [35]:
bmi_delta = ihd_bmi.merge(ihd_fpg[['ihd_fpg_rr','age_start','sex']], on=['age_start','sex']).merge(ihd_sbp[['ihd_sbp_rr','age_start','sex']], on=['age_start','sex'])
bmi_delta['fpg_delta'] = ((0.149278 * (bmi_delta['ihd_bmi_rr'] - 1)) + 1)/ bmi_delta['ihd_fpg_rr']
bmi_delta['sbp_delta'] = ((0.311516 * (bmi_delta['ihd_bmi_rr'] - 1)) + 1)/ bmi_delta['ihd_sbp_rr']
bmi_delta = bmi_delta[['ihd_bmi_rr','ihd_fpg_rr','ihd_sbp_rr','age_start', 'sex', 'fpg_delta','sbp_delta']]
bmi_delta.head()

,ihd_bmi_rr,ihd_fpg_rr,ihd_sbp_rr,age_start,sex,fpg_delta,sbp_delta
0,1.296208,1.1512,1.3402,70,Male,0.907068,0.815008
1,1.291149,1.1897,1.2947,75,Male,0.877080,0.842433
2,1.395200,1.1485,1.3821,65,Male,0.922068,0.812612
3,1.425974,1.2145,1.4584,55,Male,0.875742,0.776672
4,2.094858,1.6915,1.8558,30,Male,0.687815,0.722635


## Lastly, we find the Mediation Factors

In [36]:
rr_fpg = 1.34
rr_sbp_1 = 1.025
rr_bmi = 1.14

In [37]:
bmi_delta['fpg_delta_adj'] = bmi_delta.fpg_delta / 5 #5 BMI unit RRs to a 1 unit FPG RR  
bmi_delta['sbp_delta_adj'] = bmi_delta.sbp_delta * 2 #5 unit BMI RR to a 10 unit SBP RR
bmi_delta['MF_FPG'] = ((rr_fpg-1)*bmi_delta['fpg_delta_adj']) / (rr_bmi - 1)
bmi_delta['MF_SBP_1'] = ((rr_sbp_1-1)*bmi_delta['sbp_delta_adj']) / (rr_bmi - 1)
bmi_delta.head()

,ihd_bmi_rr,ihd_fpg_rr,ihd_sbp_rr,age_start,sex,fpg_delta,sbp_delta,fpg_delta_adj,sbp_delta_adj,MF_FPG,MF_SBP_1
0,1.296208,1.1512,1.3402,70,Male,0.907068,0.815008,0.181414,1.630016,0.440576,0.291074
1,1.291149,1.1897,1.2947,75,Male,0.877080,0.842433,0.175416,1.684865,0.426010,0.300869
2,1.395200,1.1485,1.3821,65,Male,0.922068,0.812612,0.184414,1.625224,0.447861,0.290219
3,1.425974,1.2145,1.4584,55,Male,0.875742,0.776672,0.175148,1.553343,0.425360,0.277383
4,2.094858,1.6915,1.8558,30,Male,0.687815,0.722635,0.137563,1.445270,0.334081,0.258084


In [38]:
HF_MF=pd.DataFrame(bmi_delta)
HF_MF.to_csv('/ihme/homes/lutzes/heart_failure_MF.csv')

In [28]:
bmi_delta.MF_FPG.mean()

0.4018318358576316

## Testing based on the intervention if we need FPG 

This is bit tough/estimated since right now I am extracting data from a figure. I hope to find better data at some point. I am choosing to assess this at 1 year since that is when there is the greatest change in both BMI and FPG.

In [6]:
#int_bmi = 2.5
#int_fpg = 0.3 mmol/L --> 5 mg/dL

For BMI will account for a 1/2 unit change in FPG, the intervention saw a ~5 unit change, we should add mediation 